In [1]:
# Install required libraries
!pip install streamlit --quiet
!npm install -g localtunnel
!pip install pandas mlxtend --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00

added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [2]:
!streamlit run app.py &>/dev/null&

In [3]:
!streamlit hello




  Welcome to Streamlit. Check out our demo in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.37.192:8501

  Ready to create your own Python apps super quickly?
  Head over to https://docs.streamlit.io

  May you create awesome apps!


  Stopping...
  Stopping...


In [4]:
!pip install streamlit
!pip install pyngrok

In [5]:
!pip install streamlit pandas mlxtend openpyxl localtunnel

ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [19]:

with open('market_basket_app.py', 'w') as f:
    f.write("""
import streamlit as st
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from collections import defaultdict
import openpyxl

# Data preparation function
def data_prep(dataframe):
    # Create a 'Product_ID' column based on unique Product Name
    dataframe['Product_ID'] = dataframe['Product Name'].astype('category').cat.codes + 1
    return dataframe

# Create a transaction-product matrix for Apriori Algorithm
def create_transaction_product_df(dataframe, id=False):
    if id:
        # Create a basket-like structure using Product IDs
        return dataframe.groupby(['Transaction ID', 'Product_ID'])['Product Name'].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)
    else:
        # Create a basket-like structure using Product Names
        return dataframe.groupby(['Transaction ID', 'Product Name'])['Product Name'].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)

# Run Apriori algorithm and generate association rules
def apriori_alg(dataframe, support_val=0.01):
    transaction_product_df = create_transaction_product_df(dataframe, id=True)
    frequent_itemsets = apriori(transaction_product_df, min_support=support_val, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    sorted_rules = rules.sort_values("lift", ascending=False)
    return sorted_rules

# Check for product ID given a product name
def check_product_id(dataframe, product_name):
    product_id = dataframe[dataframe["Product Name"] == product_name]["Product_ID"].unique()[0]
    return product_id, product_name

# Recommend products based on association rules from the same category
def recommend_product(dataframe, product_name, support_val=0.01):
    product_id, _ = check_product_id(dataframe, product_name)
    product_category = dataframe[dataframe['Product_ID'] == product_id]['Category'].values[0]  # Get category of the product

    sorted_rules = apriori_alg(dataframe, support_val)
    recommendation_list = []

    # Loop through the association rules to find related products from the same category
    for idx, product in enumerate(sorted_rules["antecedents"]):
        if product_id in product:
            for rec_product in sorted_rules.iloc[idx]["consequents"]:
                rec_category = dataframe[dataframe['Product_ID'] == rec_product]['Category'].values[0]
                if rec_category == product_category:  # Only recommend products from the same category
                    recommendation_list.append(rec_product)

    recommendation_list = list(dict.fromkeys(recommendation_list))  # Remove duplicates

    # Get product names for recommendations
    if recommendation_list:
        recommended_products = [dataframe[dataframe['Product_ID'] == recommendation_list[0]]['Product Name'].values[0]]  # Get one recommendation
    else:
        recommended_products = []
    return recommended_products

# Save only one product recommendation per product to an Excel file
def save_recommendations_to_excel(dataframe, sorted_rules, output_filename="Product_Recommendations.xlsx"):
    recommendations = {}

    for index, row in sorted_rules.iterrows():
        antecedents = list(row['antecedents'])
        consequents = list(row['consequents'])
        for antecedent in antecedents:
            if antecedent not in recommendations:  # Add only one recommendation per product
                # Filter by category to ensure same category recommendations
                antecedent_category = dataframe[dataframe['Product_ID'] == antecedent]['Category'].values[0]
                for consequent in consequents:
                    consequent_category = dataframe[dataframe['Product_ID'] == consequent]['Category'].values[0]
                    if consequent_category == antecedent_category:  # Ensure same category recommendation
                        recommendations[antecedent] = consequent
                        break

    # Create a DataFrame to save the recommendations
    recommendations_df = pd.DataFrame(list(recommendations.items()), columns=['Product_ID', 'Recommended Product ID'])
    recommendations_df['Product Name'] = recommendations_df['Product_ID'].apply(lambda x: dataframe[dataframe['Product_ID'] == x]['Product Name'].values[0])
    recommendations_df['Recommended Product Name'] = recommendations_df['Recommended Product ID'].apply(
        lambda id: dataframe[dataframe['Product_ID'] == id]['Product Name'].values[0])

    # Save to Excel
    recommendations_df.to_excel(output_filename, index=False)
    return output_filename

# Streamlit app interface
st.title("Market Basket Analysis and Product Recommendation")

# Upload CSV file
uploaded_file = st.file_uploader("Upload your CSV file", type=["csv"])

if uploaded_file is not None:
    data = pd.read_csv(uploaded_file)
    data = data_prep(data)  # Prepare data by adding 'Product_ID' column

    st.write("Preview of the uploaded data with Product_ID:")
    st.write(data.head())

    # Show the frequent itemsets and association rules
    if st.button("Run Apriori Algorithm"):
        frequent_itemsets = apriori_alg(data)
        st.write("Frequent Itemsets and Association Rules:")
        st.write(frequent_itemsets)

        # Save all recommendations to an Excel file
        output_filename = save_recommendations_to_excel(data, frequent_itemsets)
        st.write(f"Product recommendations have been saved to {output_filename}.")

        with open(output_filename, 'rb') as file:
            btn = st.download_button(
                label="Download Product Recommendations",
                data=file,
                file_name=output_filename,
                mime='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
            )

    # Recommendation system
    product_name = st.text_input("Enter a product name for recommendations:")

    if st.button("Get Recommendations"):
        if product_name:
            try:
                product_list = recommend_product(data, product_name)
                if len(product_list) == 0:
                    st.write("There are no products that can be recommended!")
                else:
                    st.write("Related product for product name -", product_name, ":")
                    st.write(product_list[0])  # Show only one recommendation
            except:
                st.write("The product name you entered does not exist in the dataset.")
        else:
            st.write("Please enter a valid product name.")
""")



In [7]:
!streamlit run market_basket_app.py &>/dev/null &

In [8]:
!lt --port 8501

your url is: https://mighty-lions-float.loca.lt
^C


In [9]:
!wget -q -O - https://loca.lt/mytunnelpassword

35.245.37.192

In [10]:
!ngrok authtoken 2n9Tr7RQk2RUgPLuT2dIAXi1bpt_3JGdZk7HLowT5HqjC1VTn


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
!pip install --upgrade pyngrok


In [12]:
!npm install -g localtunnel
!lt --port 8501



changed 22 packages, and audited 23 packages in 947ms

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
your url is: https://fifty-crews-lay.loca.lt
^C


In [13]:
from pyngrok import ngrok
import os


# Start a new tunnel on port 8501 (for Streamlit app)
public_url = ngrok.connect(8501, "http")
print(f"Public URL: {public_url}")

# Run the Streamlit app
os.system('streamlit run market_basket_app.py &')


Public URL: NgrokTunnel: "https://4fdc-35-245-37-192.ngrok-free.app" -> "http://localhost:8501"


0